In [1]:
import utils
import warnings

warnings.filterwarnings('ignore')
utils.set_css_style('style.css')

# 1. Hyperparameter Tuning

Choosing the right hyperparameters for your machine learning algorithm is a crucial task, since it can make a big difference on the performance of a model. 

Machine Learning models are composed of two different types of parameters:

* **Hyperparameters** = are all the parameters which can be arbitrarily set by the user before starting training (eg. Learning rate, regularization parameter, batch size in the mini-batch gradient descent...).

* Model **parameters** = are instead learned during the model training (eg. weights in Linear Regression, Neural Networks...).

The model parameters define how to use input data to get the desired output and are learned at training time. Instead, Hyperparameters determine how our model is structured in the first place.
Hyperparameter tuning is a type of optimization problem. We have a set of hyperparameters and we aim to find the right combination of their values which can help us to find either the minimum (eg. loss) or the maximum (eg. accuracy) of a function.

<img src="figures/hyperparameter-tuning.png" alt="hyperparameter-tuning" style="width: 500px;"/>


We need to tune our hyperparameters to get the best out of them. Here is a list of the most important hyperparameters in neural networks algorithms:

* Learning rate.
* Momentum beta.
* Mini-batch size.
* No. of hidden units.
* No. of layers.
* Learning rate decay.
* Regularization lambda.
* Activation functions.
* Adam beta1, beta2 & epsilon.

Its hard to decide which hyperparameter is the most important in a problem. It depends a lot on your problem.

# 2. Grid search

Grid search is a method by which we create sets of possible hyper-parameters values for each hyper-parameter, then test them against each other in a “grid.” 

Here's the workflow:

* Define a grid on n dimensions, where each of these maps for an hyperparameter. e.g. `n = (learning_rate, dropout_rate, batch_size)`
* For each dimension, define the range of possible values: e.g. `batch_size = [4, 8, 16, 32, 64, 128, 256]`
* Search for all the possible configurations and wait for the results to establish the best one: e.g. `C1 = (0.1, 0.3, 4) -> acc = 92%`, `C2 = (0.1, 0.35, 4) -> acc = 92.3%`, etc...

The image below illustrates a simple grid search on two dimensions for the Dropout and Learning rate.

<img src="figures/grid_search.png" alt="grid_search" style="width: 500px;"/>

This strategy is embarrassingly parallel because it doesn't take into account the computation history (we will expand this soon). But what it does mean is that the more computational resources 💻 you have available, then the more guesses you can try at the same time!

The real pain point of this approach is known as the curse of dimensionality. This means that more dimensions we add, the more the search will explode in time complexity (usually by an exponential factor), ultimately making this strategy unfeasible!

It's common to use this approach when the dimensions are less than or equal to 4. But, in practice, even if it guarantees to find the best configuration at the end, it's still not preferable.

# 3. Random search

As its name suggests, Random Search uses random combinations of hyperparameters. This means that not all of the parameter values are tried, and instead, parameters will be sampled with fixed numbers of iterations given by `n_iter` in the `RandomizedSearchCV()` function.

Random Search would be advised to use over Grid Search when the searching space is high meaning that there are more than 3 dimensions as Random Search is able to explore a wider hyperparameter space. In the below example, grid search only tested three unique values for each hyperperameter, whereas the random search tested 9 unique values for each. That means if one hyperparameter is more important than the others, random search will be better. Think of it this way: if hyperparameter 2 doesn’t really matter, then we would want 9 different hyperparameter 1 values to test instead of 3. The same holds true for higher dimensions (more hyperparameters).

<div class="item">
    <img src="figures/gs-vs-rs.png" alt="/gs-vs-rs" width="600px"/>
    <span class="caption">With grid search, nine trials only test three distinct places. With random search, all nine trails explore distinct values.</span>
</div>

# 4. Bayesian Optimisation

## 4.1. Bayesian Optimisation Intuition

* One of the commonest reasons for not using neural networks is that it requires a lot of skill to set hyper-parameters.
    * Number of layers
    * Number of units per layer
    * Type of unit
    * Weight penalty
    * Learning rate
    * Momentum  etc. etc.

* Naive grid search: Make a list of alternative values for each hyper-parameter and then try all possible combinations. 
    * But can we do better than this?

* Sampling random combinations: This is much better if  some hyper-parameters have no effect.
    * Its a big waste to exactly repeat the settings of the other hyper-parameters.

### Machine learning to the rescue!

* Instead of using random combinations of values for the hyper-parameters, why not look at the results so far?
    * Predict regions of the hyper-parameter space that might give better results.
    * We need to predict how well a new combination will do and also model the uncertainty of that prediction. 

* We assume that the amount of computation involved in evaluating one setting of the hyper-parameters is huge.
    * Much more than the work involved in building a model that predicts the result from knowing previous results with different settings of the hyper-parameters.

### Gaussian Process models!

* These models assume that similar inputs give similar outputs. 
    * This is a very weak but very sensible prior for the effects of hyper-parameters.
* For each input dimension, they  learn the appropriate scale for measuring similarity.
    * Is 200 similar to 300?
    * Look to see if they give similar results in the data so far.

* GP models do more than just predicting a single value.
    * They predict a Gaussian distribution of values.
* For test cases that are close to several, consistent training cases the predictions are fairly sharp.
* For test cases far from any training cases, the predictions have high variance.


### A sensible way to decide what to try

* Keep track of the best setting so far.
* After each experiment this might stay the same or it might improve if the latest result is the best.
* Pick a setting of the hyper-parameters such that the **expected improvement** in our best setting is big.

<img src="figures/expected_improvement.png" alt="expected_improvement" style="width: 400px;"/>

## 4.2. Bayesian Optimisation in Details



We want to select hyperparameters that give the best performance. This amounts to an optimization problem, specifically, the problem of optimizing a function $f(x)$ (i.e., performance as a function of hyperparameter values) over a compact set $\mathcal{A}$, which we can write mathematically as:

$$ argmax_{x \in \mathcal{A}} f(x) $$

Many optimization settings, like the one above, assume that the objective function f(x) has a known mathematical form, is convex or is cheap to evaluate. But the above characteristics do not apply to the problem of finding hyperparameters where the function is unknown and expensive to evaluate. This is where Bayesian optimization comes into play.

Bayesian optimization is an extremely powerful technique when the mathematical form of the function is unknown or expensive to compute. The main idea behind it is to compute a posterior distribution over the objective function based on the data (using the famous Bayes theorem), and then select good points to try with respect to this distribution.


### 4.2.1. Gaussian Processes

To use Bayesian optimization, we need a way to flexibly model distributions over objective functions. This is a bit trickier than modeling a distribution over, say, real numbers, since we’ll need one such distribution to represent our beliefs about $f(x)$ for each x. If x contains continuous hyperparameters, there will be infinitely many x for which we must model $f(x)$, i.e., construct a distribution for it. For this problem, Gaussian processes are a particularly elegant technique. In effect, they generalize multidimensional Gaussian distributions, and versions exist that are flexible enough to model any objective function. Originally GPs were developed to help search for gold; we use them to look for machine learning gold.

As a good visual description of what is occurring in Bayesian Optimization take a look at the images below (source). The first shows an initial estimate of the surrogate model — in black with associated uncertainty in gray — after two evaluations. Clearly, the surrogate model is a poor approximation of the actual objective function in red:

<img src="figures/bo_1.png" alt="bo_1" style="width: 500px;"/>

The next image shows the surrogate function after 8 evaluations. Now the surrogate almost exactly matches the true function. Therefore, if the algorithm selects the hyperparameters that maximize the surrogate, they will likely yield very good results on the true evaluation function.

<img src="figures/bo_2.png" alt="bo_2" style="width: 500px;"/>

### 4.2.2. Exploration-exploitation tradeoff

Once we have a model for the objective function, we need to select a good point to try. This creates an exploration-exploitation tradeoff similar to one you face when selecting a restaurant: Do you eat at your usual place (which is reliably good), or take a risk and try the new one (which might be great, or might be terrible)?

### 4.2.3. Acquisition function

To encode the tradeoff between exploration and exploitation, we can define an acquisition function that provides a single measure of how useful it would be to try any given point. We can then repeatedly find the point that maximizes this acquisition function and try it next. Here are a few possibilities:

#### Upper confidence bound

Perhaps the simplest acquisition function looks at an optimistic value for the point. Given a parameter beta, it assumes the value of the point will be beta standard deviations above the mean. Mathematically, it is

$$ \alpha(x) = \mu(x) + \beta \sigma(x) $$ 

By varying beta, we can encourage the algorithm to explore or exploit more.

#### Probability of improvement

The main idea behind probability of improvement acquisition function is that we pick the next point based on the maximum probability of improvement (MPI) with respect to the current maximum.

<img src="figures/acquisition_function.png" alt="acquisition_function" style="width: 600px;"/>

In the above figure, the maximum observed value, $y^+$ is at $x^+$. The area of the green shaded region gives the probability of improvement at $x_3$. The model predicts negligible possibility of improvement at $x_1$ or $x_2$. Whereas sampling at $x_3$ is likely to produce an improvement over $y^+$.

$$ P(x) = P(y > y^+) $$


# 5. Hyperband

Hyperband is a variation of random search, but with some explore-exploit theory to find the best time allocation for each of the configurations. It is described in details in [this paper](https://arxiv.org/pdf/1603.06560.pdf). A good introduction to this algorithm is the successive halving algorithm:
* Randomly sample 64 hyper-parameter sets in the search space.
* Evaluate after 100 iterations the validation loss of all these.
* Discard the lowest performers of them to keep only a half.
* Run the good ones for 100 iterations more and evaluate.
* Discard a half.
* Run the good ones for 200 iterations more and evaluate.
* Repeat till you have only one model left.

This algorithm needs a definition of an iteration (can be set so that the first model evaluation is done after a couple of epochs) and a total budget of iterations (which will set the total number of explored configurations).
Successive halving suffers from what is called the “n vs B/n” trade-off. B is the total budget to be allocated for hyper-parameter search, and n is the number of configurations to be explored. B/n is the average amount of resource allocated to a specific configuration.

For a given budget, it is not clear whether to search for many configurations (large n) for a small time, or explore few configurations but allocating a lot of resources to them (large B/n). For a given problem, if hyper-parameter configurations can be discriminated quickly (if the dataset converges fast, if bad configurations reveal themselves quickly, or if the hyper-parameter search space is not chosen wisely enough so that a randomly picked one is likely to be very bad), then n should be chosen large. On the contrary, if they are slow to differentiate (or if the search space is small, but the best configuration is sought with high confidence), then B/n should be large (at the expense of the number of tested configurations).

The drawbacks of these strategies are the following:
* If n is large, then some good configurations which can be slow to converge at the beginning will be killed off early.
* If B/n is large, then bad configurations will be given a lot of resources, even though they could have been stopped before.

Hyperband does away with this dilemma by considering several possible values of n for a fixed B, in essence performing a grid search over feasible value of n. In the infinite time limit, this enables to find the optimal allocation strategy.


The following table shows an example of Hyperband for different allocation strategies:
* `max_iter = 81` is the maximum iterations/epochs per configuration.
* `eta = 3` # defines downsampling rate in each allocation strategy.
* Each inner loop indexed by `s` is designed to take `B` total iterations.



```
max_iter = 81        s=4             s=3             s=2             s=1             s=0
eta = 3              n_i   r_i       n_i   r_i       n_i   r_i       n_i   r_i       n_i   r_i
B = 5*max_iter       ---------       ---------       ---------       ---------       ---------
                      81    1         27    3         9     9         6     27        5     81
                      27    3         9     9         3     27        2     81
                      9     9         3     27        1     81
                      3     27        1     81
                      1     81
```

If you wan't to know more about the algorithm, please check [this reference](https://homes.cs.washington.edu/~jamieson/hyperband.html).